# Bonus Time

We'll start with the agent we built in ex 1, but add a UiPath-hosted MCP server, so the agent can invoke a MCP tool.

There is an MCP Server created in AMER Presales folder in staging.uipath.com/uipathlabs - the URL for this srever is: https://staging.uipath.com/uipathlabs/Playground/agenthub_/mcp/4332993a-91e0-4b3c-84ca-c4b47bf6e386/uipath-genai-websummary

The MCP server contains one tool - the GenAI Activities Web Summary activity.

## First things first

Let's run some imports for libraries and set up our LLM to go through the UiPath trust layer again.

In [1]:
import os
from typing import Any
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain.agents import create_agent
from uipath_langchain.chat.models import UiPathChat
from langchain_mcp_adapters.tools import load_mcp_tools
from mcp import ClientSession
from langchain_mcp_adapters.client import MultiServerMCPClient

llm = UiPathChat(
    model="gpt-4o-2024-11-20",
    temperature=0,
    max_tokens=4000,
    timeout=30,
    max_retries=2,
    # other params...
)

llm.invoke("What is the capital of the moon?")

Response content: {"id":"chatcmpl-Cz83MjkpfOfTr8AbJYdv1Wz5OGYqu","model":"gpt-4o-2024-11-20","object":"chat.completion","choices":[{"finish_reason":"stop","index":0,"message":{"content":"The Moon does not have a capital, as it is not a country or governed by any political entity. It is a natural satellite of Earth. However, humans have explored the Moon, and notable locations include the Apollo landing sites, such as Tranquility Base, where Apollo 11 landed in 1969.","role":"assistant"}}],"created":1768685000,"usage":{"completion_tokens":64,"prompt_tokens":15,"total_tokens":79,"cache_read_input_tokens":0}}



AIMessage(content='The Moon does not have a capital, as it is not a country or governed by any political entity. It is a natural satellite of Earth. However, humans have explored the Moon, and notable locations include the Apollo landing sites, such as Tranquility Base, where Apollo 11 landed in 1969.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 15, 'total_tokens': 79, 'cache_read_input_tokens': 0}, 'model_name': 'gpt-4o-2024-11-20', 'finish_reason': 'stop', 'system_fingerprint': 'chatcmpl-Cz83MjkpfOfTr8AbJYdv1Wz5OGYqu', 'created': 1768685000}, id='lc_run--019bcdd7-561c-7281-bbce-b80dfd0337a4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 64, 'total_tokens': 79})

# Now we'll get right to business

We'll define a simple agent like before, but this time wrap the agent with an MCP client. 

This follows a simple sequence:

* Connect to MCP server
* Read the available tools
* Add those tools to the agent
* Invoke the agent

MCP allows us to decouple the tools definition from the agent or worflow code that runs the agent - this means that I can change the tools on the server without changing the code we're writing here. This separation speeds up development, since teams can independently build tool catalogs, while agent engineers can simply discover and add them to agents.

### Connect to the MCP Server with a client

The below code creates the connection so the client can discover which tools are available.

In [3]:
client = MultiServerMCPClient(
    {
        "weather": {
            "transport": "http",
            "url": "https://staging.uipath.com/uipathlabs/Playground/agenthub_/mcp/4332993a-91e0-4b3c-84ca-c4b47bf6e386/uipath-genai-websummary",
            "headers": {  
                "Authorization": "Bearer " + os.getenv("UIPATH_ACCESS_TOKEN"),  
            },  
        }
    }
)

tools = await client.get_tools()
print(tools)

[StructuredTool(name='webSummary', description='Generate AI-based summaries of web search results, synthesizing insights from multiple sources with citations.', args_schema={'type': 'object', 'properties': {'query': {'type': 'string', 'title': 'Search', 'description': 'The natural language query to search the web for'}}, 'additionalProperties': False, 'required': ['query']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7d2be5074ae0>)]


### Create the agent with the tools

In [9]:
from langchain.messages import HumanMessage

agent = create_agent(llm, tools=tools)

### Invoke the agent

result = await agent.ainvoke({"messages": [HumanMessage(content="Tell me the latest in AI Agent news")]})
print(result["messages"][-1].content)


Here are some recent developments in AI agents:

1. **LiveRamp's Agentic Orchestration Launch** (October 2025): LiveRamp introduced a platform enabling autonomous AI agents to access identity resolution and audience activation tools, supporting marketing operations. This comes amid a significant rise in agentic AI job postings.

2. **G2's AI Roundup**: Highlights include OpenAI's partnership with Bain & Company for B2B applications, ChatGPT's Search rollout for real-time information retrieval, and advancements in MarTech like Zeta Global's acquisition of LiveIntent for identity resolution.

3. **Microsoft's Copilot Studio**: Microsoft announced the public preview of Copilot Studio, which allows users to create autonomous agents. This initiative is scaling into 2025.

4. **C.H. Robinson's AI Agent Fleet Expansion**: The company expanded its fleet of AI agents, automating over 3 million tasks such as quoting and order processing.

5. **Enterprise Adoption Trends**: By 2025, 45% of enterp

# Your challenge

Take this agent and add structured input/output with the graph approach that we used in the main exercise.